Registro de los mejores makespans y número de iteraciones

## Best Move

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [58]:
# load data to dataframes
def load_data(data_directory, name):
    dfs = []

    for i in range(number_of_files):
        dfs.append([])
        file_df = pd.read_csv(data_directory + name.replace("N", str(i + 1)))
        column_names = file_df.columns.values.tolist()

        # remove trailing spances from column names
        column_names = [column_name.strip() for column_name in column_names]
        file_df.columns = column_names

        idx = [-1] + file_df[file_df['current_makespan'] == ' current_makespan'].index.tolist()

        for j in range(number_of_examples - 1):
            dfs[i].append(file_df.loc[idx[j] + 1:idx[j + 1] - 2,:])

        dfs[i].append(file_df.loc[idx[number_of_examples - 1] + 1:,:])
        
    return dfs

In [59]:
numeric_columns = ['best_makespan', 'current_makespan', 'current_temp', 'iter_number_in_temp',
                      'iter_per_temp', 'max_temperature', 'min_temperature', 'num_jobs',
                      'num_machines', 'num_operations', 'temp_change_ratio', 'worst_makespan', 'prob_acceptance']

string_columns = ['should_accept', 'move_direction', 'equal_count','first_op_moved', 'last_op_moved', 
                      'longer_count', 'operation_duration', 'selected_machine', 'shorter_count', 
                      'switch_operation_1', 'violates_constraint','moves']

def cast_types(dfs, numeric_columns, string_columns):
    # for each dataframe remove the first row from data
    for file_df in dfs:
        for df in file_df:
            df.drop(df.index[0], inplace=True)

    # for each dataframe, cast columns to appropriate types
    for file in range(number_of_files):
        for example in range(number_of_examples):
            for column in numeric_columns:
                if column in dfs[file][example].columns.values.tolist():
                    dfs[file][example].loc[:, column] = \
                            dfs[file][example].loc[:,column].apply(pd.to_numeric, errors='coerce')

            for column in string_columns:
                if column in dfs[file][example].columns.values.tolist():
                    dfs[file][example].loc[:, column] = \
                            dfs[file][example].loc[:,column].astype(str)

In [56]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

In [65]:
heuristics = ["BEST", "BEST2", "FIRST", "SEQ", "DE", "PS"]
files = ["JSSP_3_fN.csv", "JSSP_4_fN.csv", "JSSP_5_fN.csv", "JSSP_6_fN.csv", "JSSP_7_fN.csv", "JSSP_2_fN.csv"]
directory = "../../data/first_run/"
number_of_files = 7
number_of_examples = 20
all_dfs = {}
makespans_dfs = {}
iterations_dfs = {}

for heuristic, file in zip(heuristics, files):
    all_dfs[heuristic] = load_data(directory, file)
    cast_types(all_dfs[heuristic], numeric_columns, string_columns)

    makespans_dfs[heuristic] = pd.DataFrame()
    iterations_dfs[heuristic] = pd.DataFrame()
    for file in range(number_of_files):
        values = []
        iters = []
        for example in range(10):
            values.append(all_dfs[heuristic][file][example].loc[:, "best_makespan"].min())
            iters.append(all_dfs[heuristic][file][example].shape[0])
            
        column_name = "{0}x{1}:".format(int(all_dfs[heuristic][file][0].loc[0:1, "num_jobs"]), 
                                        int(all_dfs[heuristic][file][0].loc[0:1, "num_machines"]))
        
        makespans_dfs[heuristic][column_name] = pd.Series(values)
        iterations_dfs[heuristic][column_name] = pd.Series(iters)

In [66]:
for heuristic in heuristics:
    print(heuristic)
    makespans_dfs[heuristic].style.apply(highlight_min)
makespans_dfs[heuristic]

BEST
BEST2
FIRST
SEQ
DE
PS
